In [1]:
#import all the needed libraries
import pandas as pd
import numpy as np
import matplotlib as mp
import os
import re
import yaml
from subprocess import Popen


In [3]:
#fill out information about the run, fill...
fileName = "Example"
fillNumber = "001"



In [52]:
def differLine(line):
    lineSplitted = line.split()
    
    return [float(num) for num in lineSplitted]    

In [53]:
def loadDataEmit(dir):
    #this function loads emit data,argument dir is X or Y or Z

    #open data files 
    data = []
    with open(fileName + "." + dir + "emit." + fillNumber,"r", encoding="utf-8") as fileX:
        for line in fileX:
            newLine = differLine(yaml.safe_load(line +"\n"))
            data.append(newLine)
            
    return data


In [57]:
emitX = loadDataEmit("X")
emitY = loadDataEmit("Y")
emitZ = loadDataEmit("Z")
    

0.0
